In [3]:
# Seminarfacharbeit Informatik Klasse 10: Errechnung von Strukturmerkmalen organischer Verbindungen auf Basis von IR-Spektren mit Hilfe künstlicher neuronaler Netze
# Datum: 31.05.23
# Autoren: Albert-V. Meyer, Joshua Schraud, Paula Kaltwasser

# benötigte Module importieren
import numpy
import matplotlib.pyplot
import scipy.special
import time

# nimmt vom Benutzer die Pfade für den Test- und Traingsdatensatz entgegen
path_train = str(input("Bitte geben Sie den Pfand der Datei des verwendeten Trainingsdatensatzes an (nicht in Anführungszeichen schreiben)."))
path_test = str(input("Bitte geben Sie den Pfand der Datei des verwendeten Testdatensatzes an (nicht in Anführungszeichen schreiben)."))

Bitte geben Sie den Pfand der Datei des verwendeten Trainingsdatensatzes an (nicht in Anführungszeichen schreiben).C:\Users\Joshua\OneDrive\Dokumente\Seminarfach_10\Trainings- und Testdaten_IR\IR_Trainingsdatensatz_vollständig.csv
Bitte geben Sie den Pfand der Datei des verwendeten Testdatensatzes an (nicht in Anführungszeichen schreiben).C:\Users\Joshua\OneDrive\Dokumente\Seminarfach_10\Trainings- und Testdaten_IR\IR_Testdatensatz_vollständig.csv


In [4]:
# Zeitpunkt des Beginns messen
time1 = time.time()

# Definieren der Klasse für das neuronale Netz
class neuralNetwork:

    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # Knotenanzahl für Input-, Output- und versteckte Schicht defininieren
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # Gewichtsmatrizen zwischen den Schichten des neuralen Netztes
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        # Lernrate
        self.lr = learningrate

        # Definition der Sigmoid-Funktion als Aktivierungsfunktion
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    def train(self, inputs_list, targets_list):
        # Teil 1: Ausgabe für gegebenes Trainingsbeispiel erzeugen
        inputs = numpy.array(inputs_list, ndmin = 2).T
        targets = numpy.array(targets_list, ndmin = 2).T
        
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        # Teil 2: Ausgabe mit gewünschter Ausgabe vergleichen, Abweichung ermitteln und Gewichte aktualisieren

        # Fehler berechnen
        output_errors = targets - final_outputs

        # Fehler der versteckten Schicht ermitteln
        hidden_errors = numpy.dot(self.who.T, output_errors)

        # Aktalisierung der Gewichte zwischen versteckten Schicht und Ausgabeschicht
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))

        # Gewichtsaktualisierung zwischen Eingabeschicht und versteckten Schicht
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    def query(self, inputs_list):
        # Liste für Eingabe in Matrix umwandlen
        inputs = numpy.array(inputs_list, ndmin = 2).T
        # Erstellen einer Matrix für Eingänge der versteckten Schicht
        hidden_inputs = numpy.dot(self.wih, inputs)

        # Matrix für Ausgänge der Aktivierungsfunktion erstellen
        hidden_outputs = self.activation_function(hidden_inputs)

        # Matrizen für Ein- und Ausgangssignale der letzten Schicht
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

# Knotenanzahl pro Schicht festlegen
input_nodes = 7200
hidden_nodes = 200
output_nodes = 13

# Lernrate festlegen
learning_rate = 0.0001

# neuronales Netz erstellen
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

# Importieren der Datenmenge für das Training
training_data_file = open(path_train, 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# legt Anzahl der Trainingsdurchläufe (Epochen) fest
epochs = 20

for e in range(epochs):
    # jeden Teil der Trainingsdaten zur Eingabe anspassen und neuronales Netz damit trainieren
    
    # jede Zeile der CSV-Datei (die Daten jedes einzelnen Trainingsmoleküls) wird durchgearbeitet
    for record in training_data_list:
        
        # Listen für Eingabe- und Zielwerte für das Training
        inputs = []
        targets = []
        
        # Daten werden in eine Liste überführt
        all_values = record.split(',')
        values_list = []
        for i in all_values:
            values_list.append(int(i))
            
        # Eingabe- und Zieldaten werden separiert
        for i in (values_list[:13]):
            targets.append(i)
        input_values = values_list[13:]
        
        # Eingabedaten werden um einen Anfangs- und einen Endwert zur Vereinheitlichung ergänzt 
        input_values = [4000, 98] + input_values
        input_values.append(400)
        input_values.append(0)
        
        # Punkte mit ganzzahligen x-Werten der linearen Funktionen zwischen den Banden werden in eine Liste eintragen
        for j in range(len(input_values)):
            if j % 2 == 0 and input_values[j] != 400:
                x1 = input_values[j]
                y1 = input_values[j + 1]
                x2 = input_values[j + 2]
                y2 = input_values[j + 3]
                for k in range(input_values[j], input_values[j + 2], -1):
                    y = ((y2 - y1) / (x2 - x1)) * k + (y1 - ((y2 - y1) / (x2 - x1) * x1))
                    inputs.append(k / 4001)
                    inputs.append(y / 100)

        # Eingabewerte für ein Molekül werden in die allgemeine Liste für Trainingseingabewerte eingefügt 
        n.train(inputs, targets)
        pass

print("Das Training ist nun abgeschlossen.")

Das Training ist nun abgeschlossen.


In [6]:
# Testen des neuronales Netzes

# Testdatensätze importieren
test_data_file = open(path_test, 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

# Liste für Leistung des Netzes
scorecard = []

# lässt Antworten des neuronales Netzes ausgeben, wenn es auf 1 gestellt wird
print_answers = 1

# Testdatensätze in neuronales Netz einspeisen und mit richtigem Ergebnis vergleichen
for i in range(10):
    for record in test_data_list:
        all_values = record.split(',')
        correct_label = []
        for j in range(13):
            if all_values[j] == "1":
                correct_label.append(1)
            else:
                correct_label.append(0)
        if print_answers == 1:
            print("korrekte Anwort:", str(correct_label))
        
        # Liste für Eingabewerte für das Testen
        inputs = []
        
        # Daten für das Testen werden in eine Liste überführt
        all_values = record.split(',')
        values_list = []
        for i in all_values:
            values_list.append(int(i))   
        input_values = values_list[13:]
        
        # Eingabedaten werden um einen Anfangs- und einen Endwert zur Vereinheitlichung ergänzt 
        input_values = [4000, 98] + input_values
        input_values.append(400)
        input_values.append(0)
        
        # Punkte mit ganzzahligen x-Werten der linearen Funktionen zwischen den Banden werden in eine Liste eintragen
        for j in range(len(input_values)):
            if j % 2 == 0 and input_values[j] != 400:
                x1 = input_values[j]
                y1 = input_values[j + 1]
                x2 = input_values[j + 2]
                y2 = input_values[j + 3]
                for k in range(input_values[j], input_values[j + 2], -1):
                    y = ((y2 - y1) / (x2 - x1)) * k + (y1 - ((y2 - y1) / (x2 - x1) * x1))
                    inputs.append(k / 4001)
                    inputs.append(y / 100)
        
        # Ausgabe des neuronalen Netzes wird erzeugt
        outputs = n.query(inputs)
        
        # Antworten des Netzes werden in eine Liste überführt
        label = []
        for i in range(13):
            if outputs[i] > 0.5:
                label.append(1)
            else:
                label.append(0)
        
        # Ausgabe des Netzes wird zurückgegeben, wenn print_answers auf 1 gesetzt wird
        if print_answers == 1:
            print("Antwort des neuronalen Netztes:", label)
            print("---------------------------------")
        
        # Ausgaben des Netzes und korrekte Antworten werden verglichen
        for k in range(13):
            if label[k] == correct_label[k]:
                scorecard.append(1)
            else:
                scorecard.append(0)
        pass
    pass

# Leistung des neuronalen Netztes berechnen
scorecard_array = numpy.asarray(scorecard)
print("Das Testen des neuronalen Netztes ist jetzt abgeschlossen.")
print ("Leistung = ", scorecard_array.sum() / scorecard_array.size * 100, "%")

# Zeit messen und ausgeben
time2 = time.time()
timespan = time2 - time1
print("Trainings- und Testdauer: ", timespan)

korrekte Anwort: [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Antwort des neuronalen Netztes: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---------------------------------
korrekte Anwort: [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]
Antwort des neuronalen Netztes: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
---------------------------------
korrekte Anwort: [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Antwort des neuronalen Netztes: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---------------------------------
korrekte Anwort: [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0]
Antwort des neuronalen Netztes: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
---------------------------------
korrekte Anwort: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Antwort des neuronalen Netztes: [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
---------------------------------
korrekte Anwort: [1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]
Antwort des neuronalen Netztes: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---------------------------------
korrekte Anwort: [1, 0